### Text generation with RNN example;

* This code is mostly stolen from this website. Reference:https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [37]:
#importing the classes and functions that we needb for training our model

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [38]:
#dowloaded the text and put it into the same directory with python program
#Read the data, turn it into lower case
data = open("alice_in_wonderland.txt","r",encoding='utf-8').read().lower()


#creating a set of all of the distinct characters in the text
#This get the set of characters used in the data and sorts them
# represent each character with a unique integer
chars = sorted(list(set(data)))

#Total number of characters used in the data
totalChars = len(data)

#Number of unique chars
numberOfUniqueChars = len(chars)



print(totalChars)     #the text has more than 150.000 characters
print(numberOfUniqueChars)  #number of distinct characters more than alphabet 26

163816
60


In [20]:
#This allows for characters to be represented by numbers
CharsForids = {char:Id for Id, char in enumerate(chars)}

#This is the opposite to the above
idsForChars = {Id:char for Id, char in enumerate(chars)}

#How many timesteps? or how many characters we want to process in one go
numberOfCharsToLearn = 100


In [33]:
#Since our timestep sequence represents a process for every 100 chars we omit
#the first 100 chars so the loop runs a 100 less or there will be index out of
#range
counter = totalChars - numberOfCharsToLearn

#Inpput data
charX = []
#output data
y = []
#This loops through all the characters in the data skipping the first 100
for i in range(0, counter, 1):
    #This one goes from 0-100 so it gets 100 values starting from 0 and stops
    #just before the 100th value
    theInputChars = data[i:i+numberOfCharsToLearn]
    #With no : you start with 0, and so you get the actual 100th value
    #Essentially, the output Chars is the next char in line for those 100 chars
    #in X
    theOutputChars = data[i + numberOfCharsToLearn]
    #Appends every 100 chars ids as a list into X
    charX.append([CharsForids[char] for char in theInputChars])
    #For every 100 values there is one y value which is the output
    y.append(CharsForids[theOutputChars])


###### First we must transform the list of input sequences into the form [samples, time steps, features] expected by an LSTM network.
###### Next we need to rescale the integers to the range 0-to-1 to make the patterns easier to learn by the LSTM network that uses the sigmoid activation function by default.
###### Finally, we need to convert the output patterns (single characters converted to integers) 

In [34]:
X = np.reshape(charX, (len(charX), numberOfCharsToLearn, 1))
#Len charX represents how many of those time steps we have
#Our features are set to 1 because in the output we are only predicting 1 char
#Finally numberOfCharsToLearn is how many character we process


#This is done for normalization
X = X/float(numberOfUniqueChars)

#This sets it up for us so we can have a categorical(#feature) output format
y = np_utils.to_categorical(y)
#print(y)

### Define the Model

In [7]:
model = Sequential()
#Since we know the shape of our Data we can input the timestep and feature data
#The number of timestep sequence are dealt with in the fit function
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))     #LSTM layer with 256 memory units
model.add(Dropout(0.2))   #A Dropout layer to prevent overfitting to the training data


#number of features on the output
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=5, batch_size=128)

Epoch 1/5
1280/1280 [==============================] - 827s 616ms/step - loss: 3.0980
Epoch 2/5
1280/1280 [==============================] - 457s 357ms/step - loss: 2.8475
Epoch 3/5
1280/1280 [==============================] - 440s 343ms/step - loss: 2.7560
Epoch 4/5
1280/1280 [==============================] - 422s 330ms/step - loss: 2.6780
Epoch 5/5
1280/1280 [==============================] - 361s 282ms/step - loss: 2.6200


There is no test dataset. We are modeling the entire training dataset to learn the probability of each character in a sequence.

##### finally, make predictions
  *  ###### pick a random input pattern as our seed sequence, then print generated characters as we generate them.

In [43]:
randomVal = np.random.randint(0, len(charX)-1)
randomStart = charX[randomVal]
print("".join([idsForChars[value] for value in randomStart]))



 ‘or perhaps they won’t walk the way i want
to go! let me see: i’ll give them a new pair of boots ev


In [45]:
for i in range(1000):
    
    x = np.reshape(randomStart, (1, len(randomStart), 1))
    x = x/float(numberOfUniqueChars)
    pred = model.predict(x, verbose=0)
    index = np.argmax(pred)
    randomStart.append(index)
    randomStart = randomStart[1: len(randomStart)]
    
print("".join([idsForChars[value] for value in randomStart]))




o the tooee to the tooee to the tooee to the tooee to the tooee to the tooee to the tooee to the too


I couldnt be able to generate a meaningful sentence text. 

#### as a summary wa can list the steps of our approach like;
* Convert text from list of strings into list of lists of integers (sequences)
* Create feature and labels from sequences
* Build LSTM model with LSTM, and Dense layers
* Train model to predict next work in sequence
* Make predictions by passing in starting sequence